In [216]:
import pandas as pd
import os
import numpy as np

In [217]:
data = pd.read_parquet(".." + os.sep + "data" + os.sep +
                "dati_presenze_politici.parquet")


trasformazione delle date in oggetti datetime

In [218]:
data['DATA'] = data['DATA'].apply(lambda x: x.replace(" 00:00",""))
data['DATA'] = data['DATA'].apply(lambda x: x.replace(":00",""))
data.DATA = pd.to_datetime(data["DATA"],  format="%d.%m.%Y")


In [219]:
data.head(4)

,CANALE,PROGRAMMA,DATA,COGNOME,NOME,MICRO_CATEGORIA,ARGOMENTO,DURATA,TIPO_TEMPO
2,CANALE 5,TG5,2022-01-01,Berlusconi,Silvio,PDL - Forza Italia,Politica e attività istituzionali,42,Notizia
3,CANALE 5,TG5,2022-01-01,Calenda,Carlo,Azione - +Europa - Radicali Italiani,Politica e attività istituzionali,6,Notizia
4,CANALE 5,TG5,2022-01-01,Conte,Giuseppe,Movimento 5 Stelle,Politica e attività istituzionali,36,Notizia
5,CANALE 5,TG5,2022-01-01,D'Amato,Alessio,Partito Democratico,Cronaca,3,Notizia


filtro per soggetti collettivo

In [220]:
dati_soggetti_colletivi = data[data.NOME == "Soggetto Collettivo"]
#dati_soggetti_colletivi = dati_soggetti_colletivi[dati_soggetti_colletivi.NOME ==
#                                        "Soggettivo Collettivo"]

In [221]:
minuti_soggetti_collettivi = dati_soggetti_colletivi.groupby(["COGNOME"])["DURATA"].sum().reset_index().fillna(0).sort_values(by="DURATA", ascending=False)

944052

In [298]:
out = "data: ["
for idx,row in minuti_soggetti_collettivi.iterrows():
    name = row.COGNOME
    value = round(row.DURATA / 60,2)   
    #out += '{name: "%s", value: %s},' % (name, value)
    minuti_soggetti_collettivi_tipo = dati_soggetti_colletivi[dati_soggetti_colletivi.COGNOME == "Partito Democratico"].groupby(["TIPO_TEMPO"])[
        "DURATA"].sum().reset_index().fillna(0).sort_values(by="DURATA", ascending=False)
    notizia = minuti_soggetti_collettivi_tipo[minuti_soggetti_collettivi_tipo['TIPO_TEMPO']
                                              == "Notizia"].DURATA.values[0]
    parola = minuti_soggetti_collettivi_tipo[minuti_soggetti_collettivi_tipo['TIPO_TEMPO']
                                             == "Parola"].DURATA.values[0]
    out += '{name: "%s", value: %s,' % (name, value)
    out += 'children: [{'
    out += 'name: "notizia", value: %s},' % round(notizia / 60,2)
    out += '{ name: "parola", value: %s}' % round(parola / 60, 2)
    out += "]},"
out = out[:-1] + "]"


In [299]:
out

'data: [{name: "Movimento 5 Stelle", value: 25002.78,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Partito Democratico", value: 15734.58,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Lega Nord", value: 9565.58,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "PDL - Forza Italia", value: 7237.23,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Fratelli d\'Italia Centrodestra Nazionale", value: 3294.9,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Italia Viva", value: 3272.5,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Liberi e Uguali", value: 1848.77,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Sardine", value: 286.1,children: [{name: "notizia", value: 15734.2},{ name: "parola", value: 0.38}]},{name: "Forza Nuova", va

filtro per soli politici

In [224]:
dati_politici = data[data.NOME != "Soggettivo Collettivo"]
dati_politici = dati_politici[dati_politici.NOME != "Soggetto Collettivo"]

creazione campo NOME_COGNOME

In [225]:
dati_politici['NOME_COGNOME'] = dati_politici['NOME'] + \
    " " + dati_politici['COGNOME']


calcolo presenze per politico

In [226]:
presenze = dati_politici.pivot_table(index="NOME_COGNOME",
                             values="DURATA", aggfunc=np.sum)#

In [227]:
presenze.reset_index(inplace=True)


In [228]:
presenze.sort_values(by="DURATA", ascending=False, inplace=True)

In [229]:
totale_politici = len(dati_politici.NOME_COGNOME.unique())


In [230]:
totale_politici


8547

ricerca più presenti nell'ultimo decile

In [231]:
#ultimo decile
p = presenze.DURATA.quantile(0.9)

In [232]:
presenze_politici_rilevanti = presenze[(presenze.DURATA > presenze.DURATA.quantile(0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


di questi prendiamo l'ultimo decile

In [233]:
presenze_politici_top_rilevanti = presenze[(presenze.DURATA > presenze_politici_rilevanti.DURATA.quantile(
    0.9)) & (presenze.DURATA <= presenze.DURATA.max())]


In [234]:
presenze_politici_top_rilevanti.shape[0]

86

In [235]:
presenze_politici_top_rilevanti

,NOME_COGNOME,DURATA
5887,Matteo Salvini,4830594
7830,Silvio Berlusconi,2542977
5001,Luigi Di Maio,2460683
5880,Matteo Renzi,1896863
3719,Giorgia Meloni,1835590
...,...,...
6929,Pietro Grasso,167791
5412,Maria Elena Boschi,165889
3641,Gianni Cuperlo,165194
4933,Luciano Nobili,160188


In [236]:
distribuzione_argomenti = data.groupby(['DATA', 'ARGOMENTO','TIPO_TEMPO'])[
    'DURATA'].aggregate('sum').unstack().reset_index()

In [237]:
distribuzione_argomenti.fillna(0, inplace=True)

In [238]:
distribuzione_argomenti['TOTALE'] = distribuzione_argomenti['Notizia'] + distribuzione_argomenti['Parola']

In [239]:
distribuzione_argomenti


TIPO_TEMPO,DATA,ARGOMENTO,Notizia,Parola,TOTALE
0,2018-01-01,Costume e società,16.0,0.0,16.0
1,2018-01-01,Cronaca,154.0,404.0,558.0
2,2018-01-01,"Economia, finanza e lavoro",11.0,0.0,11.0
3,2018-01-01,Giustizia,261.0,247.0,508.0
4,2018-01-01,Mass media e spettacolo,0.0,46.0,46.0
...,...,...,...,...,...
13197,2022-06-30,Costume e società,0.0,290.0,290.0
13198,2022-06-30,Cronaca,20.0,159.0,179.0
13199,2022-06-30,"Economia, finanza e lavoro",188.0,2706.0,2894.0
13200,2022-06-30,Esteri,109.0,1226.0,1335.0


In [240]:
totale_notizie = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Notizia'].aggregate('sum').to_frame().reset_index()

out = "["
for idx, row in totale_notizie.iterrows():
    v = round(row.Notizia / 60,2)
    out += str(v) + ","
out = out[:-1] + "]"


In [241]:
totale_parole = distribuzione_argomenti.sort_values(by="ARGOMENTO", ascending=True).groupby(
    'ARGOMENTO')['Parola'].aggregate('sum').to_frame().reset_index()

out = "["
for idx, row in totale_parole.iterrows():
    v = round(row.Parola / 60, 2)
    out += str(v) + ","
out = out[:-1] + "]"


In [242]:
out

'[3683.68,1972.52,800.98,25861.02,3272.42,54273.38,12913.32,0.42,2530.72,331.0,1374.03,10432.68,742280.92,109.47,0.25,226.5,10249.23,975.97]'

In [243]:
minuti_argomenti = distribuzione_argomenti.pivot_table(index="ARGOMENTO", values="TOTALE", aggfunc=np.sum).reset_index().sort_values(by="ARGOMENTO", ascending=True)

In [244]:
out = "radar: { indicator: ["
for idx, row in minuti_argomenti.iterrows():
    name = row.ARGOMENTO
    value = round(row.TOTALE / 60,2)
    out += "{ name: '%s', value: %s}," % (name, value)
out = out[:-1] + "]}"

In [245]:
volume_notizie_parole = data.groupby(['DATA', 'TIPO_TEMPO'])['DURATA'].aggregate('sum').unstack().reset_index()

In [246]:
volume_notizie_parole.sort_values(by="DATA", ascending=True, inplace=True)

In [247]:
volume_notizie_parole


TIPO_TEMPO,DATA,Notizia,Parola
0,2018-01-01,8543,5404
1,2018-01-02,13613,8908
2,2018-01-03,18222,16568
3,2018-01-04,17561,19757
4,2018-01-05,16868,14548
...,...,...,...
1637,2022-06-26,3858,9586
1638,2022-06-27,15647,35442
1639,2022-06-28,12438,28493
1640,2022-06-29,8713,31095


In [248]:
volume_notizie_parole['TOTALE'] = volume_notizie_parole['Notizia'] + volume_notizie_parole['Parola']

In [249]:
volume_notizie_parole[volume_notizie_parole.TOTALE == volume_notizie_parole.TOTALE.max()]


TIPO_TEMPO,DATA,Notizia,Parola,TOTALE
1114,2021-01-19,18186,163020,181206


In [250]:
volume_notizie_parole[volume_notizie_parole.TOTALE ==
                      volume_notizie_parole.TOTALE.min()]


TIPO_TEMPO,DATA,Notizia,Parola,TOTALE
61,2018-03-03,463,883,1346


In [251]:
out = ""
for idx,row in volume_notizie_parole.iterrows():
    day = row.DATA.strftime('%Y/%m/%d')
    parole = round(row.Parola / 60,2)
    notize = round(row.Notizia /60,2)
    out += "['%s', %s, 'INTERVENTI']," % (day, parole)
    out += "['%s', %s, 'NOTIZIE']," % (day, notize)


In [252]:
data.COGNOME.unique()

array(['Berlusconi', 'Calenda', 'Conte', ..., 'Delli Colli', 'Calearo',
       'Gramazio'], dtype=object)